In [28]:
import faiss
import numpy as np
from langchain.embeddings import HuggingFaceEmbeddings
import pymongo


def text_embed(query):
    embedding_query = embeddings.embed_query(query)
    return np.array([embedding_query])


class fs:

    def __init__(self, raw, top_k=-1):
        self.top_k = top_k
        self.index = faiss.IndexFlatL2(768)
        self.index.add(raw)

    def faiss_similar(self, embedding):
        top_k = self.index.ntotal
        scores, indices = self.index.search(embedding, top_k)
        scores = scores[0].tolist()
        indices = indices[0].tolist()
        self.scores = dict(zip(indices, scores))

    
    # def tag_similar(self, tags_embedding):
    #     tags_embedding

query = ""
embeddings = HuggingFaceEmbeddings()   
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["photoshop"]
collection = db["pexel_photo"]


raw_data = [(i["title"], i["tags"], i["url"]) for i in collection.find({})]
raw_title = embeddings.embed_documents([i[0] for i in raw_data])
raw_title = np.array(raw_title)
# tag_faiss = fs(raw_tag)
title_faiss = fs(raw_title)


def search(query, topk=6):
    embedding_query = text_embed(query)
    # tag_faiss.faiss_similar(embedding_query)
    title_faiss.faiss_similar(embedding_query)
    res = sorted(title_faiss.scores.items(), key=lambda x: x[1], reverse=True)
    top_res = [i[0] for i in res][: topk]
    return [raw_data[i][2] for i in top_res]

In [29]:
idx = search("女人")

In [30]:
[raw_data[i][2] for i in idx]

['https://images.pexels.com/photos/4998009/pexels-photo-4998009.jpeg?cs=srgb&dl=pexels-rachel-claire-4998009.jpg&fm=jpg',
 'https://images.pexels.com/photos/2330169/pexels-photo-2330169.jpeg?cs=srgb&dl=pexels-bruno-salvadori-2330169.jpg&fm=jpg',
 'https://images.pexels.com/photos/10677607/pexels-photo-10677607.jpeg?cs=srgb&dl=pexels-ron-lach-10677607.jpg&fm=jpg',
 'https://images.pexels.com/photos/10910449/pexels-photo-10910449.jpeg?cs=srgb&dl=pexels-%D0%B3%D0%B0%D0%BB%D0%B8%D0%BD%D0%B0-%D0%BB%D0%B0%D1%81%D0%B0%D0%B5%D0%B2%D0%B0-10910449.jpg&fm=jpg',
 'https://images.pexels.com/photos/3932961/pexels-photo-3932961.jpeg?cs=srgb&dl=pexels-tatiana-syrikova-3932961.jpg&fm=jpg',
 'https://images.pexels.com/photos/3975654/pexels-photo-3975654.jpeg?cs=srgb&dl=pexels-tatiana-syrikova-3975654.jpg&fm=jpg']